## DATA 620 - Project 1

---

### Mael Illien, Jeremy OBrien & Vanita Thompson

* Identify a large 2-node network dataset—you can start with a dataset in a repository. Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.
* Reduce the size of the network using a method such as the island method described in chapter 4 of social network analysis.
* What can you infer about each of the distinct groups?

You may work in a small group on the project.

Your code and analysis should be delivered in an IPython Notebook by end of day Sunday.

## Introduction

## Setup

In [14]:
import csv
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite as bp
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from IPython.display import display, HTML
%matplotlib inline

## Data Import

In [15]:
# ISS data example - likely too few nodes for assignment

#iss_data = pd.read_csv('IS_ALLEGIANCES_2M.csv')
#iss_data.info()

In [40]:
# digg dataset test

digg_data = pd.read_csv('digg.edges')
digg_data.rename(columns={'1':'user_id', '2':'story_id', '1214799565':'timestamp'}, inplace=True)
digg_data.sort_values(by=['user_id','story_id'], ascending=True).head(n=20)

,user_id,story_id,timestamp
0,1,3,1230047012
1,1,4,1194104160
2,1,5,1199180573
3,1,6,1196465462
1731652,1,279631,1246573330
1740475,1,279646,1246577372
1747181,1,279662,1246577864
1900734,1,279841,1246437965
2040916,1,280026,1246336555
2070779,1,280053,1246336400


## Data Processing

In [41]:
G = nx.Graph()

In [42]:
# Add user nodes
users = list(set(digg_data['user_id']))
G.add_nodes_from(users, bipartite=0)

# Add story nodes
stories = list(set(digg_data['story_id']))
G.add_nodes_from(stories, bipartite=1)

# Add votes as edges
votes = list(zip(digg_data['user_id'], digg_data['story_id']))
G.add_edges_from(votes)

In [43]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 283183
Number of edges: 4559024
Average degree:  32.1984


## Island Method

In [44]:
list(G.nodes(data=True))

[(1, {'bipartite': 1}),
 (2, {'bipartite': 1}),
 (3, {'bipartite': 1}),
 (4, {'bipartite': 1}),
 (5, {'bipartite': 1}),
 (6, {'bipartite': 1}),
 (7, {'bipartite': 1}),
 (8, {'bipartite': 1}),
 (9, {'bipartite': 1}),
 (10, {'bipartite': 1}),
 (11, {'bipartite': 1}),
 (12, {'bipartite': 1}),
 (13, {'bipartite': 1}),
 (14, {'bipartite': 1}),
 (15, {'bipartite': 1}),
 (16, {'bipartite': 1}),
 (17, {'bipartite': 1}),
 (18, {'bipartite': 1}),
 (19, {'bipartite': 1}),
 (20, {'bipartite': 1}),
 (21, {'bipartite': 1}),
 (22, {'bipartite': 1}),
 (23, {'bipartite': 1}),
 (24, {'bipartite': 1}),
 (25, {'bipartite': 1}),
 (26, {'bipartite': 1}),
 (27, {'bipartite': 1}),
 (28, {'bipartite': 1}),
 (29, {'bipartite': 1}),
 (30, {'bipartite': 1}),
 (31, {'bipartite': 1}),
 (32, {'bipartite': 0}),
 (33, {'bipartite': 1}),
 (34, {'bipartite': 1}),
 (35, {'bipartite': 1}),
 (36, {'bipartite': 1}),
 (37, {'bipartite': 1}),
 (38, {'bipartite': 1}),
 (39, {'bipartite': 1}),
 (40, {'bipartite': 1}),
 (41, {'b

In [52]:
# G.degree
# G.nodes()
deg_userstory, deg_storyuser = bp.degrees(G, [0,1])

In [68]:
user_degrees = [val for (node, val) in sorted(deg_userstory(), key=lambda pair: pair[0])]

In [75]:
print('user nodes:', len(users))
print('story nodes:', len(stories))
G_top = {} 
print(len(deg_userstory))
print(len(deg_storyuser))
print(len(user_degrees))

user nodes: 166632
story nodes 261770
283182
1
283182


In [33]:
# Function to trim edges non-destrucively based on threshold edge weigh
def trim_edges(graph, weight=1):
    graph_trimmed = net.Graph()
    for u,v,edata in graph.edges(data=True):
        if edata['weight'] > weight:
            graph_trimmed.add_edge(u,v,edata)  # u and v are nodes on edges, edata is edge data 
    return graph_trimmed

def island_method(G, iterations=5):
    weights = [edata['weight'] for u,v,edata in G.edges(data=True)]  # list of weights for all edges
    weight_min = int(min(weights))  # lowest weight
    weight_max = int(max(weights))  # highest weight
    step = int((weight_max - weight_min) / iterations)  # size of the step as function of range and iterations
    return[[threshold, trim_edges(G, threshold)] for threshold in range(weight_min, weight_max, step)]

G_islands = island_method(G)

KeyError: 'weight'

In [26]:
# graph_pos = nx.spring_layout(G, k=.5)

MemoryError: Unable to allocate 597. GiB for an array with shape (283187, 283187) and data type float64